install dependecies 

In [ ]:
!pip install pyTelegramBotAPI yfinance pandas numpy ta

configure with your telegram bot token and chat id 

In [ ]:
import yfinance as yf
import telebot
import pandas as pd
import numpy as np
from ta.trend import ADXIndicator, MACD
from ta.momentum import StochasticOscillator
import time

TELEGRAM_BOT_TOKEN = 'TELEGRAMTOKEN'
CHAT_ID = '-CHATID' 
bot = telebot.TeleBot(TELEGRAM_BOT_TOKEN)

SYMBOLS = {'XAU/USD': 'GC=F', 'NDX/USD': '^NDX'}
TIMEFRAMES = {'1h': '1h', '15m': '15m'}  

def get_data(symbol, timeframe, limit=100):
    yf_symbol = SYMBOLS.get(symbol, symbol)
    print(f"[INFO] Récupération des données réelles pour {symbol} en {timeframe}")
    df = yf.download(yf_symbol, period='7d', interval=TIMEFRAMES[timeframe])
    df = df.tail(limit)
    df.reset_index(inplace=True)
    
    df = df[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']]
    df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df = df.dropna()
    return df

def calculate_indicators(df):
    if df.empty:
        return df  
    
    # ADXr
    adx = ADXIndicator(high=df['high'], low=df['low'], close=df['close'], window=14)
    df['adx'] = adx.adx()
    df['adxr'] = (df['adx'] + df['adx'].shift(1)) / 2
    
    # MACD
    macd = MACD(close=df['close'])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    df['macd_cross'] = np.where(df['macd'] > df['macd_signal'], 1, -1)
    
    # Center of Gravity (COG)
    df['cog'] = df['close'].rolling(window=10).mean()
    return df

def check_signals(df):
    if df.shape[0] < 2:
        return False 
    
    latest = df.iloc[-1]
    previous = df.iloc[-2]
    
    macd_cross_up = previous['macd_cross'] == -1 and latest['macd_cross'] == 1
    adxr_up = latest['adxr'] > previous['adxr']
    cog_cross = previous['close'] < previous['cog'] and latest['close'] > latest['cog']
    
    if macd_cross_up and adxr_up and cog_cross:
        return True
    return False

# Fonction principale
def main():
    while True:
        for symbol in SYMBOLS.keys():
            for timeframe in TIMEFRAMES.keys():
                df = get_data(symbol, timeframe)
                df = calculate_indicators(df)
                
                if not df.empty and check_signals(df):
                    message = f'Signal détecté sur {symbol} en {timeframe} !\nADXR, MACD et COG alignés.'
                    bot.send_message(CHAT_ID, message)
                    print(message)
                else:
                    print(f"[INFO] Aucun signal détecté pour {symbol} en {timeframe}")
        
        time.sleep(10)  

if __name__ == '__main__':
    main()


Dont forget to redo the mapping if not using yfinance 